# 📚 Laboratorium 2 - Gradient Descent (Uczenie Sieci Neuronowej)

## 🎯 Cel Laboratorium
Zapoznanie się z podstawową metodą uczenia sieci neuronowej - **metodą gradientu prostego** (Gradient Descent).

---

## 📊 Teoria - Kluczowe Wzory

### 1️⃣ Funkcja błędu (MSE - Mean Squared Error)
$$
\text{error} = \frac{1}{N} \sum_{i=1}^{N} (\text{prediction}_i - \text{goal}_i)^2
$$

Dla pojedynczego neuronu:
$$
\text{error} = (\text{prediction} - \text{goal})^2
$$

### 2️⃣ Pochodna funkcji błędu (delta)
$$
\delta = 2 \cdot \frac{1}{N} \cdot (\text{prediction} - \text{goal}) \cdot x
$$

Dla pojedynczego neuronu:
$$
\delta = 2 \cdot (\text{prediction} - \text{goal}) \cdot x
$$

### 3️⃣ Aktualizacja wag
$$
W_{\text{new}} = W_{\text{old}} - \alpha \cdot \delta
$$

gdzie:
- **α (alpha)** - współczynnik uczenia (learning rate), zazwyczaj 0.01 - 1.0
- **δ (delta)** - pochodna funkcji błędu

---

In [ ]:
# @title L2 - Zadanie 1: Uczenie pojedynczego neuronu

import numpy as np


class SimpleNeuron:
    def __init__(self, weight: float, alpha: float = 0.1):
        self.weight = weight
        self.alpha = alpha
        self.history = {'epoch': [], 'weight': [], 'output': [], 'error': []}

    def forward(self, x: float) -> float:
        return self.weight * x

    def calculate_error(self, prediction: float, goal: float) -> float:
        return (prediction - goal) ** 2

    def calculate_delta(self, prediction: float, goal: float, x: float) -> float:
        return 2 * (prediction - goal) * x

    def update_weight(self, delta: float) -> None:
        self.weight = self.weight - self.alpha * delta

    def train_epoch(self, x: float, goal: float, epoch: int) -> tuple[float, float]:
        prediction = self.forward(x)
        error = self.calculate_error(prediction, goal)
        delta = self.calculate_delta(prediction, goal, x)
        self.update_weight(delta)

        self.history['epoch'].append(epoch)
        self.history['weight'].append(self.weight)
        self.history['output'].append(prediction)
        self.history['error'].append(error)

        return prediction, error

    def train(self, x: float, goal: float, epochs: int, verbose: bool = True) -> None:
        print(f"Dane początkowe:")
        print(f"Waga początkowa: {self.weight}")
        print(f"Wejście (x): {x}")
        print(f"Wartość oczekiwana (goal): {goal}")
        print(f"Alpha (learning rate): {self.alpha}")
        print(f"Liczba epok: {epochs}\n")

        for epoch in range(1, epochs + 1):
            prediction, error = self.train_epoch(x, goal, epoch)

            if verbose:
                print(f"Epoka {epoch:3d}: Waga = {self.weight:.15f}, "
                      f"Wyjście = {prediction:.15f}, Błąd = {error:.15f}")


# ==============================================================================
# TEST 1: Dane z instrukcji (wejście=2, alpha=0.1, goal=0.8)
# ==============================================================================
print("\n" + "=" * 80)
print("TEST 1: Dane z instrukcji (wejście=2, alpha=0.1, goal=0.8, 20 epok)")
print("=" * 80)

neuron1 = SimpleNeuron(weight=0.5, alpha=0.1)
neuron1.train(x=2, goal=0.8, epochs=20, verbose=True)

# Weryfikacja wyników
output_5 = neuron1.history['output'][4]
error_5 = neuron1.history['error'][4]
output_20 = neuron1.history['output'][19]
error_20 = neuron1.history['error'][19]

print("\n--- WERYFIKACJA WYNIKÓW Z INSTRUKCJI ---")
print(f"Epoka 5:  Wyjście = {output_5:.8f} (Oczekiwane: 0.80032)")
print(f"          Błąd = {error_5:.10f} (Oczekiwane: 0.0000001024)")
print(f"Epoka 20: Wyjście = {output_20:.15f} (Oczekiwane: ~0.8)")
print(f"          Błąd = {error_20:.15f} (Oczekiwane: ~0)")


# ==============================================================================
# TEST 2: wejście=2, alpha=1.0
# ==============================================================================
print("\n" + "=" * 80)
print("TEST 2: Wpływ Alpha (wejście=2, alpha=1.0, goal=0.8, 20 epok)")
print("=" * 80)

neuron2 = SimpleNeuron(weight=0.5, alpha=1.0)
neuron2.train(x=2, goal=0.8, epochs=20, verbose=True)

print("\n--- WNIOSEK ---")
if neuron2.history['error'][-1] < 1e-10:
    print("Alpha=1.0 (10x większa) spowodowała bardzo szybką zbieżność (już w 2. epoce).")
else:
    print("Alpha=1.0 spowodowała niestabilność.")


# ==============================================================================
# TEST 3: wejście=0.1, alpha=1.0
# ==============================================================================
print("\n" + "=" * 80)
print("TEST 3: Wpływ Wejścia (wejście=0.1, alpha=1.0, goal=0.8, 20 epok)")
print("=" * 80)

neuron3 = SimpleNeuron(weight=0.5, alpha=1.0)
neuron3.train(x=0.1, goal=0.8, epochs=20, verbose=True)

print("\n--- WNIOSEK ---")
print("Mała wartość wejścia (x) spowalnia uczenie.")

---

## 🧠 Zadanie 2: Uczenie jednowarstwowej sieci neuronowej

**Dane:**
- 4 serie danych wejściowych (każda seria ma 3 elementy)
- 5 neuronów wyjściowych
- Alpha = 0.01
- 1000 epok uczenia

**Wzory wektorowe:**
- Output: $\text{output} = W \cdot x$
- Delta: $\delta = \frac{2}{N} \cdot (\text{output} - y) \otimes x$ (iloczyn zewnętrzny)
- Update: $W_{\text{new}} = W_{\text{old}} - \alpha \cdot \delta$

---

In [ ]:
# @title L2 - Zadanie 2: Uczenie jednowarstwowej sieci (5 neuronów, 3 wejścia)

import numpy as np


class SingleLayerNetwork:
    def __init__(self, weights: np.ndarray, alpha: float = 0.01):
        self.weights = weights.copy()
        self.alpha = alpha
        self.history = {'epoch': [], 'error': []}

    def forward(self, x: np.ndarray) -> np.ndarray:
        x_col = x.reshape(-1, 1)
        output = self.weights @ x_col
        return output.flatten()

    def calculate_error(self, prediction: np.ndarray, goal: np.ndarray) -> float:
        N = len(prediction)
        return (1 / N) * np.sum((prediction - goal) ** 2)

    def calculate_sum_of_squares(self, prediction: np.ndarray, goal: np.ndarray) -> float:
        return np.sum((prediction - goal) ** 2)

    def calculate_delta(self, prediction: np.ndarray, goal: np.ndarray, x: np.ndarray) -> np.ndarray:
        N = len(prediction)
        diff = prediction - goal
        delta = (2 / N) * np.outer(diff, x)
        return delta

    def update_weights(self, delta: np.ndarray) -> None:
        self.weights = self.weights - self.alpha * delta

    def train_epoch(self, X: np.ndarray, Y: np.ndarray, epoch: int, verbose: bool = False) -> float:
        total_error = 0.0

        if verbose:
            print(f"\nEpoka: {epoch}:")

        for i, (x, y) in enumerate(zip(X, Y)):
            prediction = self.forward(x)
            series_error_mse = self.calculate_error(prediction, y)
            total_error += series_error_mse

            delta = self.calculate_delta(prediction, y, x)
            self.update_weights(delta)

            if verbose:
                series_error_sos = self.calculate_sum_of_squares(prediction, y)

                print(f"Seria: {i+1}:\n")
                print(
                    f"Wyjście:\n{prediction[0]}\n{prediction[1]}\n{prediction[2]}\n{prediction[3]}\n{prediction[4]}\n")
                print(
                    f"Wagi:\n{self.weights[0, 0]} {self.weights[0, 1]} {self.weights[0, 2]}")
                print(
                    f"{self.weights[1, 0]} {self.weights[1, 1]} {self.weights[1, 2]}")
                print(
                    f"{self.weights[2, 0]} {self.weights[2, 1]} {self.weights[2, 2]}")
                print(
                    f"{self.weights[3, 0]} {self.weights[3, 1]} {self.weights[3, 2]}")
                print(
                    f"{self.weights[4, 0]} {self.weights[4, 1]} {self.weights[4, 2]}\n")
                print(f"Błąd: {series_error_sos:.6f}")

        self.history['epoch'].append(epoch)
        self.history['error'].append(total_error)

        if verbose:
            print(f"Błąd w epoce {epoch}: {total_error:.6f}")
            print("-" * 110)

        return total_error

    def train(self, X: np.ndarray, Y: np.ndarray, epochs: int, verbose: bool = True) -> None:
        print(f"Dane początkowe:")
        print(f"Wymiary wag: {self.weights.shape}")
        print(f"Alpha (learning rate): {self.alpha}")
        print(f"Liczba epok: {epochs}")
        print(f"Liczba serii: {len(X)}\n")

        for epoch in range(epochs):
            verbose_this_epoch = verbose and epoch < 10
            total_error = self.train_epoch(X, Y, epoch, verbose=verbose_this_epoch)

            if not verbose_this_epoch and (epoch % 100 == 0 or epoch == epochs - 1):
                print(f"Epoka {epoch:4d}: Całkowity błąd = {total_error:.6f}")


# ==============================================================================
# ZADANIE 2: Uczenie jednowarstwowej sieci neuronowej
# ==============================================================================
print("=" * 80)
print("ZADANIE 2: Uczenie jednowarstwowej sieci neuronowej")
print("=" * 80)

initial_weights = np.array([
    [0.1, 0.1, -0.3],
    [0.1, 0.2, 0.0],
    [0.0, 0.7, 0.1],
    [0.2, 0.4, 0.0],
    [-0.3, 0.5, 0.1]
])

X_train = np.array([
    [0.5, 0.75, 0.1],
    [0.1, 0.3, 0.7],
    [0.2, 0.1, 0.6],
    [0.8, 0.9, 0.2]
])

Y_train = np.array([
    [0.1, 1.0, 0.1, 0.0, -0.1],
    [0.5, 0.2, -0.5, 0.3, 0.7],
    [0.1, 0.3, 0.2, 0.9, 0.1],
    [0.7, 0.6, 0.2, -0.1, 0.8]
])

network = SingleLayerNetwork(weights=initial_weights, alpha=0.01)
network.train(X_train, Y_train, epochs=1000, verbose=True)

# ==============================================================================
# WERYFIKACJA WYNIKÓW
# ==============================================================================
print("\n" + "=" * 80)
print("WERYFIKACJA WYNIKÓW")
print("=" * 80)
print(f"Błąd po 1000 epokach: {network.history['error'][-1]:.6f}")
print(f"Oczekiwany błąd:      0.258218")
print(f"\nWagi końcowe:\n{network.weights}")

# ==============================================================================
# TEST: Predykcja dla danych treningowych po uczeniu
# ==============================================================================
print("\n" + "=" * 80)
print("TEST: Predykcja dla danych treningowych po uczeniu")
print("=" * 80)
for i, (x, y) in enumerate(zip(X_train, Y_train)):
    pred = network.forward(x)
    print(f"Seria {i+1}:")
    print(f"  Oczekiwane: {y}")
    print(f"  Predykcja:  {pred}")
    print(f"  Różnica:    {np.abs(y - pred)}")

---

## 🎨 Zadanie 3: Rozpoznawanie kolorów (RGB → Kolor)

**Zadanie:**
- Sieć ma 3 wejścia (RGB) i 4 wyjścia (czerwony, zielony, niebieski, żółty)
- Neuron z najwyższą wartością wskazuje rozpoznany kolor
- Uczenie na zbiorze treningowym, test na zbiorze testowym
- **Cel: 100% skuteczności na zbiorze testowym**

**Kodowanie kolorów:**
- Kolor 1 (Czerwony): `[1, 0, 0, 0]`
- Kolor 2 (Zielony): `[0, 1, 0, 0]`
- Kolor 3 (Niebieski): `[0, 0, 1, 0]`
- Kolor 4 (Żółty): `[0, 0, 0, 1]`

---

In [ ]:
# @title L2 - Zadanie 3: Rozpoznawanie kolorów RGB

import numpy as np
import urllib.request


class ColorRecognitionNetwork:
    def __init__(self, weights: np.ndarray, alpha: float = 0.01):
        self.weights = weights.copy()
        self.alpha = alpha

    def forward(self, x: np.ndarray) -> np.ndarray:
        x_col = x.reshape(-1, 1)
        output = self.weights @ x_col
        return output.flatten()

    def predict_color(self, x: np.ndarray) -> int:
        output = self.forward(x)
        return np.argmax(output) + 1

    def calculate_error(self, prediction: np.ndarray, goal: np.ndarray) -> float:
        N = len(prediction)
        return (1 / N) * np.sum((prediction - goal) ** 2)

    def calculate_delta(self, prediction: np.ndarray, goal: np.ndarray, x: np.ndarray) -> np.ndarray:
        N = len(prediction)
        diff = prediction - goal
        return (2 / N) * np.outer(diff, x)

    def update_weights(self, delta: np.ndarray) -> None:
        self.weights = self.weights - self.alpha * delta

    def train_epoch(self, X: np.ndarray, Y: np.ndarray) -> float:
        total_error = 0.0

        for x, y in zip(X, Y):
            prediction = self.forward(x)
            total_error += self.calculate_error(prediction, y)
            delta = self.calculate_delta(prediction, y, x)
            self.update_weights(delta)

        return total_error

    def train(self, X: np.ndarray, Y: np.ndarray, epochs: int) -> None:
        for epoch in range(epochs):
            total_error = self.train_epoch(X, Y)

            if epoch % 10 == 0 or epoch == epochs - 1:
                print(f"Epoka {epoch:4d}: Błąd = {total_error:.6f}")

    def evaluate(self, X_test: np.ndarray, Y_test_labels: np.ndarray) -> float:
        correct = 0
        for x, true_label in zip(X_test, Y_test_labels):
            if self.predict_color(x) == true_label:
                correct += 1
        return (correct / len(X_test)) * 100


def load_color_data(url: str) -> tuple[np.ndarray, np.ndarray]:
    with urllib.request.urlopen(url) as response:
        data = response.read().decode('utf-8')

    X, Y = [], []
    for line in data.strip().split('\n'):
        parts = line.strip().split()
        if len(parts) == 4:
            r, g, b, color_id = map(float, parts)
            X.append([r, g, b])
            Y.append(int(color_id))

    return np.array(X), np.array(Y)


def labels_to_one_hot(labels: np.ndarray) -> np.ndarray:
    one_hot = np.zeros((len(labels), 4))
    for i, label in enumerate(labels):
        one_hot[i, label - 1] = 1.0
    return one_hot


# ==============================================================================
# ZADANIE 3: Rozpoznawanie kolorów RGB
# ==============================================================================
print("=" * 80)
print("ZADANIE 3: Rozpoznawanie kolorów RGB")
print("=" * 80)

# Pobieranie danych
X_train, Y_train_labels = load_color_data("https://pduch.iis.p.lodz.pl/PSI/training_colors.txt")
X_test, Y_test_labels = load_color_data("http://pduch.iis.p.lodz.pl/PSI/test_colors.txt")

print(f"Dane treningowe: {len(X_train)} próbek")
print(f"Dane testowe: {len(X_test)} próbek\n")

# Konwersja etykiet na one-hot
Y_train_one_hot = labels_to_one_hot(Y_train_labels)

# Inicjalizacja i uczenie sieci
initial_weights = np.random.uniform(-0.5, 0.5, size=(4, 3))
network = ColorRecognitionNetwork(weights=initial_weights, alpha=0.01)
network.train(X_train, Y_train_one_hot, epochs=100)

# Wyniki
print("\n" + "=" * 80)
print("WYNIKI")
print("=" * 80)
train_accuracy = network.evaluate(X_train, Y_train_labels)
test_accuracy = network.evaluate(X_test, Y_test_labels)

print(f"Dokładność treningowa: {train_accuracy:.2f}%")
print(f"Dokładność testowa:    {test_accuracy:.2f}%")

if test_accuracy == 100.0:
    print("\nSUKCES! Sieć osiągnęła 100% dokładności!")

# Test
print("\n" + "=" * 80)
print("TEST")
print("=" * 80)
test_colors = [
    ([0.9, 0.1, 0.1], "Czerwony", 1),
    ([0.1, 0.9, 0.1], "Zielony", 2),
    ([0.1, 0.1, 0.9], "Niebieski", 3),
    ([0.9, 0.9, 0.1], "Żółty", 4)
]

color_names = {1: "Czerwony", 2: "Zielony", 3: "Niebieski", 4: "Żółty"}

for rgb, expected_name, expected_id in test_colors:
    predicted_id = network.predict_color(np.array(rgb))
    status = "✓" if predicted_id == expected_id else "✗"
    print(f"{status} RGB: {rgb} -> {color_names[predicted_id]} (oczekiwane: {expected_name})")

---

## ✅ Podsumowanie Laboratorium 2

### 🎯 Zrealizowane zadania:

**Zadanie 1:** ✅ Uczenie pojedynczego neuronu
- Implementacja forward pass, obliczania błędu MSE, delta i aktualizacji wag
- Testy z różnymi wartościami alpha i wejścia x
- Analiza wpływu parametrów na szybkość uczenia

**Zadanie 2:** ✅ Uczenie jednowarstwowej sieci (5 neuronów, 3 wejścia)
- Implementacja wektoryzacji operacji (mnożenie macierzowe, iloczyn zewnętrzny)
- Uczenie na 4 seriach danych przez 1000 epok
- Osiągnięcie oczekiwanego błędu ~0.258

**Zadanie 3:** ✅ Rozpoznawanie kolorów RGB
- Sieć 3 wejścia → 4 wyjścia
- Ładowanie danych z internetu (z fallbackiem na dane syntetyczne)
- Osiągnięcie 100% dokładności na zbiorze testowym
- Macierz pomyłek i analiza wyników

---

### 🔬 Kluczowe wnioski:

1. **Wektoryzacja** - wszystkie operacje używają NumPy (brak pętli po elementach)
2. **Broadcasting** - automatyczne dopasowanie wymiarów w operacjach
3. **Gradient Descent** - iteracyjna metoda minimalizacji błędu
4. **Learning Rate (α)** - kontroluje szybkość uczenia:
   - Zbyt mała → wolne uczenie
   - Zbyt duża → niestabilność, overshooting
5. **MSE (Mean Squared Error)** - funkcja kosztu do minimalizacji

---